Scrapping (университеты)

In [ ]:
!pip install requests beautifulsoup4 lxml selenium
!apt-get update
!apt-get install -y chromium-browser

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,856 kB]
Fetched 4,244 kB in 3s (1,356 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import time # для задержек между запросами (чтобы не блокировал сайт)
from typing import Dict, List, Optional
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd

In [ ]:
class UnipageScraper:

    def __init__(self, delay: float = 1.5):
        self.base_url = "https://www.unipage.net"
        self.delay = delay
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36' # чтоб не заблокали
        })

    # загрузка страницы (по умолчанию request, если сработала ошибка (мало информации или в дальнейшем не нашел карточку университета), то пробуем вытащить данные, используя библиотеку selenium)
    def get_page(self, url: str, retries: int = 3) -> Optional[BeautifulSoup]: # хотим получить html-код страницы (пробуем 3 раза)
        for attempt in range(retries):
            try:
                 # вначале через request
                response = self.session.get(url, timeout=30)
                response.raise_for_status() # если сайт выдает ошибку (например, 404), то сразу переходим к блоку - except

                html = response.text # сохраняем html код страницы
                if len(html) < 500 or "University" not in html: # если страница пустая или слишком мало инфы- пробуем через Selenium
                    raise ValueError("Пробуем через Selenium") # вызываем ошибку в таком случае

                time.sleep(self.delay)
                return BeautifulSoup(html, 'html.parser')

            # если что-то не так: (сообщает об ошибке, но продолжает работать)
            except Exception as e:
                print(f"[WARN] Ошибка при загрузке {url}: {e}")

                if attempt == retries - 1: # если это была последняя попытка - переходим к selenium

                    # настройка драйвера
                    opt = Options()
                    opt.add_argument("--headless=new") # без окна
                    opt.add_argument("--no-sandbox")
                    opt.add_argument("--disable-dev-shm-usage")
                    opt.add_argument("--disable-gpu")
                    opt.add_argument("--window-size=1920,1080")

                    # запуск драйвера
                    driver = webdriver.Chrome(options=opt)
                    try:
                        driver.get(url)
                        time.sleep(2.5)
                        html = driver.page_source
                        return BeautifulSoup(html, "html.parser")
                    finally:
                        driver.quit()

                time.sleep(5)
        return None # если все попытки не удались - None

    # доп ф-ия (извлечь число из текста, например, для рейтинга)
    def extract_number(self, text: str) -> str:
        match = re.search(r'[\d,]+', text) # ищем посл-сть любых цифры 0-9 (\d) и запятые
        return match.group(0) if match else ""

    # парсинг универа на главной странице (name, country, location (city), ranking_preview: QS, THE и ARWU, стоимость (degree type(bachelor, master и doctor)))
    def parse_main_page_card(self, card) -> Dict:
        data = {}

        # название университета и ссылка
        title_elem = card.select_one('.generated-card__title a') # ищем первый тег (select_one()) <a>, который находится внутри элемента с классом .generated-card__title
        if title_elem:
            data['name'] = title_elem.get_text(strip=True) # достаем название между тегами (без отступов)
            data['url'] = self.base_url + title_elem.get('href', '') # достаём ссылку из атрибута тега <href> и соединяем
        else:
            data['name'] = ""
            data['url'] = ""

        # cтрана (по флагу)
        flag_elem = card.select_one('.flag') # ищем все элем с классом .flag
        if flag_elem:
            flag_class = [c for c in flag_elem.get('class', []) if c.startswith('flag-')] # извлекаем список классов этого элем и отбираем только те, которе начинаются с 'flag-'
            data['country'] = flag_class[0].replace('flag-', '').upper() if flag_class else "" # теперь для каждого в списке удаляем префикс 'flag-', получаем страну и записываем в вер[нем регистре
        else:
            data['country'] = ""

        # город или локация
        location_spans = card.select('.generated-card__row span') # ищем все элем с классом .generated-card__row span
        if len(location_spans) >= 2:
            data['location'] = location_spans[1].get_text(strip=True) # на странице есть несколько <span> и второй из них содержит город, поэтому мы берём location_spans[1]
        else:
            data['location'] = ""

        # рейтинги QS, THE, ARWU
        data['qs_ranking_preview'] = ""
        data['the_ranking_preview'] = ""
        data['arwu_ranking_preview'] = ""

        tags = card.select('.tag_secondary') # ищем все элем с классом .tag_secondary
        for tag in tags:
            text = tag.get_text(strip=True)
            # исп ф-ию, которая описана выше, чтоб вытащить цифры
            if 'QS' in text:
                data['qs_ranking_preview'] = self.extract_number(text)
            elif 'THE' in text:
                data['the_ranking_preview'] = self.extract_number(text)
            elif 'ARWU' in text:
                data['arwu_ranking_preview'] = self.extract_number(text)

        # cтоимость обучения
        data['bachelor_from'] = ""
        data['bachelor_to'] = ""
        data['master_from'] = ""
        data['master_to'] = ""
        data['doctorate_from'] = ""
        data['doctorate_to'] = ""

        tuition_cards = card.select('.content-card_inverted') # элем с классом .content-card_inverted, где указана стоимость
        for tuition_card in tuition_cards: # перебираем все карточки
            dt = tuition_card.select_one('dt') # ищем элемент <dt> - в нём написано для какой степени (бакалавр, магистр и тд)
            if not dt:
                continue

            degree_type = dt.get_text(strip=True) # извлекаем из <dt> текст (удаляя лишние пробелы)
            dds = tuition_card.select('dd .content-card-number__number') # ищем все элем с классом .content-card-number__number (cписок) внутри тега <dd>

            if degree_type == 'Bachelor' and len(dds) >= 2:
                data['bachelor_from'] = dds[0].get_text(strip=True)
                data['bachelor_to'] = dds[1].get_text(strip=True)
            elif degree_type == 'Master' and len(dds) >= 2:
                data['master_from'] = dds[0].get_text(strip=True)
                data['master_to'] = dds[1].get_text(strip=True)
            elif degree_type == 'Doctorate' and len(dds) >= 2:
                data['doctorate_from'] = dds[0].get_text(strip=True)
                data['doctorate_to'] = dds[1].get_text(strip=True)

        return data

    # парсинг страницы каждого универа (location full, establishment year, students (кол-во), international students, rating - QS, THE и USA, female students, international students, acceptance rate)
    def parse_university_page(self, url: str) -> Dict:
        soup = self.get_page(url) # возвращаем объект BeautifulSoup (HTML-страница)
        if not soup:
            return {}

        data = {}

        infographic_cards = soup.select('.infographic-card__main') # вся нужная нам инфа в элементах с классом .infographic-card__main
        for card in infographic_cards: # перебор
            dt = card.select_one('dt.infographic-card__secondary-text') # название показателя
            dd = card.select_one('dd.infographic-card__primary-text') # его значение

            if dt and dd: # еслм оба есть, то берем текст без лишних пробелов
                key = dt.get_text(strip=True)
                value = dd.get_text(strip=True)

                if key == 'Location':
                    data['location_full'] = value
                elif key == 'Establishment year':
                    data['establishment_year'] = value
                elif key == 'Students':
                    data['total_students'] = value
                elif key == 'International students':
                    data['international_students'] = value
                elif key == 'Female students':
                    data['female_students'] = value
                elif key == 'Acceptance rate':
                    data['acceptance_rate'] = value

        # теперь чекаем рейтинги
        data['qs_rating'] = ""
        data['the_rating'] = ""
        data['rating_usa'] = ""

        rating_buttons = soup.select('.chart-update-button') # вся нужная нам инфа в элементах с классом .chart-update-button
        for button in rating_buttons: # перебор
            title_elem = button.select_one('.content-card__title') # показатель
            number_elem = button.select_one('.content-card-number__number') # значение

            if title_elem and number_elem:
                title = title_elem.get_text(strip=True)
                number = number_elem.get_text(strip=True)

                if 'Rating QS' in title:
                    data['qs_rating'] = number
                elif 'Rating THE' in title:
                    data['the_rating'] = number
                elif 'Rating in the USA' in title:
                    data['rating_usa'] = number

        return data

    # cбор списков универов (перебираем страницы, пока не найдем нужное кол-во (10к+), и сохраняем в список ссылки карточек универов)
    def scrape_universities_list(self, base_list_url: str, max_universities: int = 50) -> List[Dict]: # (по умолчанию 50, но мы потом говорим свое значение)
        universities = []
        page = 1

        # перебор страниц, пока не соберем нужное кол-во универов
        while len(universities) < max_universities:
            if page == 1:
                url = base_list_url
            else:
                url = f"{base_list_url}?page={page}"

            print(f"\nЗагрузка страницы {page}: {url}")
            soup = self.get_page(url) # через get_page() берём HTML страницы

            if not soup:
                print(f"Не удалось загрузить страницу {page}")
                break

            cards = soup.select('.generated-card') # получаем список всех карточек на странице

            if not cards:
                print(f"На странице {page} не найдено карточек. Завершение.")
                break

            print(f"Найдено карточек на странице {page}: {len(cards)}")

            for card in cards:
                if len(universities) >= max_universities:
                    break

                # теперь для каждой карточки вызываем парсер:
                uni_data = self.parse_main_page_card(card)
                if uni_data.get('url'):
                    universities.append(uni_data)

            print(f"Всего собрано университетов: {len(universities)}")

            if len(universities) >= max_universities:
                break

            page += 1

        return universities

    # полный сбор всех данных (главная ф-ия, она вызывает остальные в этом классе)
    def scrape_all(self, list_url: str, output_file: str = 'universities.csv',
                   max_universities: int = 50, save_every: int = 25):
        print("Начинаем сбор ссылок на университеты...")

        # сначала вызываем scrape_universities_list - список словарей, где у каждого универа будут уже данные (которые с главной страницы)
        universities = self.scrape_universities_list(list_url, max_universities)

        print(f"\nБудет обработано университетов: {len(universities)}")

        all_data = []

        # проходимся по универам
        # enumerate(iterable, start) — даёт и номер (i) (начинаем с 1), и сам объект (uni)
        for i, uni in enumerate(universities, 1):
            print(f"\n[{i}/{len(universities)}] Обработка: {uni['name']}") # Пример: [1/50] Обработка: Oxford University

            try:
                # теперь вызываем parse_university_page, откуда для каждого уника уже достаются более подробные данные
                detailed_data = self.parse_university_page(uni['url'])
                full_data = {**uni, **detailed_data} # **uni - берём все пары ключ–значение из словаря uni, ан-но **detailed_data и всё объединяем в один новый словарь full_data (одинаковые ключи перезаписались автоматически как в uni)
                all_data.append(full_data)

                filled_fields = len([v for v in full_data.values() if v]) # просто посмотреть сколько получилось вытащить признаков по конркетному универу (сколько заполнено столбцов)
                print(f"  [OK] Собрано полей: {filled_fields}")

                # на всякий случай промежуточное сохранение
                if i % save_every == 0:
                    self.save_to_csv(all_data, f'backup_{output_file}')
                    print(f"  [SAVE] Промежуточное сохранение: {i} записей")

            # если же произошла ошибка: (чтоб если что программа не упала, а перешла сюда)
            except Exception as e:
                print(f"  [ERROR] Ошибка при обработке: {e}")
                all_data.append(uni) # обрабатываем след уник

        self.save_to_csv(all_data, output_file) # вызов ф-ии, чтоб сохранить
        print(f"\n[COMPLETE] Данные сохранены в {output_file}")
        print(f"[COMPLETE] Всего университетов: {len(all_data)}")

        return all_data

    # сохранение
    def save_to_csv(self, data: List[Dict], filename: str):
      if not data:
        print("Нет данных для сохранения")
        return
      df = pd.DataFrame(data)
      df.to_csv(filename, index=False, encoding='utf-8-sig')
      print(f"[SAVE] Сохранено в {filename}")

In [ ]:
LIST_URL = "https://www.unipage.net/en/universities"

scraper = UnipageScraper(delay=1.5)

data = scraper.scrape_all(
    list_url=LIST_URL,
    output_file='universities_data.csv',
    max_universities=10000,
    save_every=25
)

Начинаем сбор ссылок на университеты...

Загрузка страницы 1: https://www.unipage.net/en/universities
Найдено карточек на странице 1: 10
Всего собрано университетов: 10

Будет обработано университетов: 10

[1/10] Обработка: Harvard University
  [OK] Собрано полей: 22

[2/10] Обработка: Massachusetts Institute of Technology
  [OK] Собрано полей: 22

[3/10] Обработка: Stanford University
  [OK] Собрано полей: 22

[4/10] Обработка: University of Cambridge
  [OK] Собрано полей: 21

[5/10] Обработка: California Institute of Technology
  [OK] Собрано полей: 22

[6/10] Обработка: University of Oxford
  [OK] Собрано полей: 21

[7/10] Обработка: Princeton University
  [OK] Собрано полей: 20

[8/10] Обработка: University of Chicago
  [OK] Собрано полей: 17

[9/10] Обработка: University College London
  [OK] Собрано полей: 18

[10/10] Обработка: ETH Zürich
  [OK] Собрано полей: 20
[SAVE] Сохранено в universities_data.csv

[COMPLETE] Данные сохранены в universities_data.csv
[COMPLETE] Всего универ